# 本セクションの目次
1. Avroフォーマット
2. 前方互換と後方互換と完全互換
3. メッセージキューとAvroを連携してみよう
4. Avroファイルの読み書き
5. Avroで前方互換をやってみよう

In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2を使っています。)。

# ビッグデータの世界のDDL

ビッグデータの世界でのDDLはRDSと同じ様にDDL文を実行することが可能です。  
今回は以下のDDLについてみていきましょう  

- Create Database 文
- CREATE EXTERNAL TABLE文
- CREATE VIEW
- ADD PARTITION(MSCK REPAIR)

## CREATE DATABASE文
データベースの作成を行います。
こちらはRDSなどのCreate Database　と同じ方法で作成が可能です。

In [ ]:
spark.sql("create database if not exists super_crush_course")

In [ ]:
# データベースの一覧を見てみましょう
spark.sql("show databases").show(truncate=False)

## CREATE EXTERNAL TABLE文
テーブル定義の構成要素をみていきましょう

```
CREATE EXTERNAL TABLE IF NOT EXISTS super_crush_course.csv_table ( id INT, date STRING)
PARTITIONED BY (dt INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION '/home/pyspark/super_crush_course.db/csv_table/dataset/parquet/';

#S3などであれば、以下のように設定を変えることも可能です。
LOCATION 's3://data.platform/super_crush_course.db/raw_zone/sampletable/';

```

ビッグデータの世界では、実データとデータベース定義/テーブル定義(メタデータ)は明確に分離されています。  
今回のコースだと実データはローカル端末のSSD(HDD)でテーブル定義はMysqlに登録されています。  
明確に分離されているからこそ、場所を指し示す宣言であるLocationが必要になってきます  

ロケーションは「super_crush_course.db」とDB名.db/TABLE名とすることが通例です。  
Externalは外部のという意味で、オンプレ環境の場合はつけないことが多かったが、クラウド環境ではつけるのが必須となっている設定。

メタデータについてさらに詳しく知りたい方は、以下の記事を参照してみてください  
「【PythonとSparkで始めるデータマネジメント入門】 ビッグデータレイクのための統合メタデータ管理入門」

上記の例は、CSV形式のテーブルです。  
それ以外にもParquet形式、Avro形式でテーブルを作成することができます。

```
# パーティションつきのテーブル
CREATE TABLE IF NOT EXISTS super_crush_course.parquet_table 
(code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
TBLPROPERTIES ("parquet.compression"="SNAPPY");
LOCATION '/home/pyspark/super_crush_course.db/parquet_table';

```

圧縮形式と保存するファイルフォーマットを指定してテーブルの作成を行なっていきます。  
ちなみにテーブル定義のカラムは、今回読み込んだCSV/JSONのスキーマになります。  
Partitionとはデータの区切りのことで、kenmeiを区切りとしてデータを保存しています(次のレクチャーにて)。　 

```
# パーティションなしのテーブルの場合
CREATE TABLE IF NOT EXISTS super_crush_course.parquet_table_with_no_partition
(code String, gengo String,kenmei, Stringwareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
STORED AS PARQUET
TBLPROPERTIES ('parquet.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/parquet_table_with_no_partition'

```

In [ ]:
## テーブル作成
# テーブル作成も同様に、spark.sqlを使ってテーブルを作成していきます。
# ロケーションはセクション2で出力したディレクトリになります。

# Parquetテーブルの作成(パーティションあり)
spark.sql("""

CREATE TABLE IF NOT EXISTS super_crush_course.parquet_table_with_partition 
(code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
TBLPROPERTIES ('parquet.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/parquet_table_with_partition'

""")

# Paruqetテーブルの作成(パーティションなし)
spark.sql("""

CREATE TABLE IF NOT EXISTS super_crush_course.parquet_table_with_no_partition 
(code String, gengo String,wareki String,kenmei String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
STORED AS PARQUET
TBLPROPERTIES ('parquet.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/parquet_table_with_no_partition'

""")

In [ ]:
# AVROテーブルの作成(パーティションあり)
spark.sql("""

CREATE TABLE IF NOT EXISTS super_crush_course.avro_table_with_partition 
(code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
PARTITIONED BY (kenmei String)
STORED AS AVRO
TBLPROPERTIES ('parquet.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/parquet_table_with_partition'

""")

# AVROテーブルの作成(パーティションなし)
spark.sql("""

CREATE TABLE IF NOT EXISTS super_crush_course.avro_table_with_no_partition 
(code String, gengo String,wareki String,kenmei String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)
STORED AS AVRO
TBLPROPERTIES ('parquet.compression'='SNAPPY')
LOCATION '/home/pyspark/super_crush_course.db/avro_table_with_no_partition'

""")

In [ ]:
# 作成したテーブルを見てみましょう
spark.sql("show tables in super_crush_course").show(truncate=False)

# ADD PRTITION
パーティションを認識するためにコマンドを発行する必要があります。

- Add Partiton
- MSCK repair Table

の2つをみていきましょう

In [ ]:
# パーティションも管理されている
spark.sql("show partitions super_crush_course.parquet_table_with_partition").show(n=2)
# 本来件名があってほしいが。。。

In [ ]:
# パーティションがテーブルは。。？クエリしてみる
spark.sql("select * from super_crush_course.parquet_table_with_no_partition").show(n=2)

In [ ]:
# パーティションがあってadd paritionをしていないテーブルは。。？クエリしてみる
spark.sql("select * from super_crush_course.parquet_table_with_partition").show(n=2)
# データを見ることができない

In [ ]:
# パーティションを追加してみる
# パーティションの追加には２種類存在しています
# add partition
# msck repair table名
spark.sql("alter table super_crush_course.parquet_table_with_partition add if not exists  partition (kenmei='東京都')")

In [ ]:
# パーティションも管理されている
spark.sql("show partitions super_crush_course.parquet_table_with_partition").show(n=20)

In [ ]:
# 再度検索を行ってみる
# パーティションがあってadd paritionをしていないテーブルは。。？クエリしてみる
spark.sql("select * from super_crush_course.parquet_table_with_partition").show()
# 追加した東京都のデータだけ見える

In [ ]:
# 一個づつAdd partitionするのは面倒なのでmsckを使う(ただし時間がかかる場合が多いので、日々の処理であればadd partitionを選択する方がいい)
spark.sql("msck repair table super_crush_course.parquet_table_with_partition")
spark.sql("msck repair table super_crush_course.avro_table_with_partition")

# Create View
ビューを作成します。
ビューとは、仮想的なテーブルのことでデータを生成しなくてもテーブルを生成することが可能です。

言葉で伝えるより実際に見た方がいいと思うので、早速作ってみましょう。　　

手っ取り早くクエリを簡単にしたい場合に有効です。

In [ ]:
# create view

spark.sql("""

create view parquet_view (gengo)
as 
select gengo from 
super_crush_course.parquet_table_with_partition

""")

In [ ]:
spark.sql("select * from parquet_view").show()

# テンポラリテーブル
テンポラリーテーブルとはデータフレームから一時的にテーブルを作成することで、SparkSessionごとに生成が可能です。

特にスキーマオンリードで読み込んだdataframeを一時的にテーブルにすることで、SQLでの操作を可能にすることができます。

一連の流れを見てみましょう

In [ ]:
# テンポラリーテーブルの作成
json_df=spark.read.json("./dataset/jinko.json")
json_df.createOrReplaceTempView("json_tmp")

In [ ]:
#　json_tempの検索
spark.sql("select * from json_tmp where kenmei='東京都'").show()

# ビッグデータ世界のDMLとは

ビッグデータの世界のSQLは基本的にSQL’ライク’です。  
というのもRDSのSQLを前提にしてライクと言っているだけなので、ビッグデータ世界を中心としたらSQLそのものです。  
RDSでのSQLに慣れている人は、ビッグデータの世界のSQLは難なくこなすことができると思います。  

- SELECT
- CTAS
- SELECT INSERT
- INSERT?
- UPDATE?
- DELETE?

今回は4つのDMLを確認してみます。

In [ ]:
# SELECT
spark.sql("select * from super_crush_course.parquet_table_with_no_partition").show()

## パーティションつきのテーブルを検索してみる
spark.sql("select * from super_crush_course.parquet_table_with_partition where kenmei ='東京都'").show()

# パーティションなしの場合は、すべてのデータを走査してから絞り込みます
# パーティションありの場合は、特定のパーティション配下のディレクトリのみをチェックします

# 大体のRDSのSQLでできることは実行可能です。

# CTAS
Create Table As Selectの略です。

簡単にいうと、Selectの返却結果からテーブルを作成することが可能です。

In [87]:
# CTASを動かしてみます

# SQLでやる方法
spark.sql(""" 
CREATE EXTERNAL TABLE if not exists super_crush_course.ctas_sql 
    STORED AS PARQUET LOCATION '/home/pyspark/super_crush_course.db/ctas_sql' 
AS
SELECT *
    FROM super_crush_course.parquet_table_with_no_partition
WHERE 1=1

""")

DataFrame[]

In [93]:
spark.sql("show tables in super_crush_course").show(truncate=False)

+------------------+-------------------------------+-----------+
|namespace         |tableName                      |isTemporary|
+------------------+-------------------------------+-----------+
|super_crush_course|avro_table_with_no_partition   |false      |
|super_crush_course|avro_table_with_partition      |false      |
|super_crush_course|ctas_dataframe                 |false      |
|super_crush_course|ctas_sql                       |false      |
|super_crush_course|parquet_table_with_no_partition|false      |
|super_crush_course|parquet_table_with_partition   |false      |
|                  |json_tmp                       |true       |
+------------------+-------------------------------+-----------+



In [101]:
json_df.write.format("parquet").mode("overwrite").saveAsTable("super_crush_course.ctas_dataframe",path='/home/pyspark/super_crush_course.db/ctas_dataframe')

22/01/06 03:11:03 WARN FileUtils: File file:/home/pyspark/pyspark_super_crush_course/spark-warehouse/super_crush_course.db/ctas_dataframe does not exist; Force to delete it.
22/01/06 03:11:03 ERROR FileUtils: Failed to delete file:/home/pyspark/pyspark_super_crush_course/spark-warehouse/super_crush_course.db/ctas_dataframe


In [104]:
spark.sql("select * from super_crush_course.ctas_dataframe").printSchema()
spark.sql("show create table super_crush_course.ctas_dataframe").show(truncate=False)

root
 |-- chu: string (nullable = true)
 |-- code: string (nullable = true)
 |-- gengo: string (nullable = true)
 |-- jinko_female: string (nullable = true)
 |-- jinko_male: string (nullable = true)
 |-- kenmei: string (nullable = true)
 |-- seireki: string (nullable = true)
 |-- sokei: string (nullable = true)
 |-- wareki: string (nullable = true)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                                                                                                                                                                                   |


In [96]:
spark.sql("select * from super_crush_course.ctas_dataframe").show()

+----+----+-----+------------+----------+--------+-------+--------+------+
| chu|code|gengo|jinko_female|jinko_male|  kenmei|seireki|   sokei|wareki|
+----+----+-----+------------+----------+--------+-------+--------+------+
|null|  00| 大正|    27918868|  28044185|    全国|   1920|55963053|     9|
|null|  01| 大正|     1114861|   1244322|  北海道|   1920| 2359183|     9|
|null|  02| 大正|      375161|    381293|  青森県|   1920|  756454|     9|
|null|  03| 大正|      424471|    421069|  岩手県|   1920|  845540|     9|
|null|  04| 大正|      476459|    485309|  宮城県|   1920|  961768|     9|
|null|  05| 大正|      444855|    453682|  秋田県|   1920|  898537|     9|
|null|  06| 大正|      490597|    478328|  山形県|   1920|  968925|     9|
|null|  07| 大正|      689225|    673525|  福島県|   1920| 1362750|     9|
|null|  08| 大正|      688272|    662128|  茨城県|   1920| 1350400|     9|
|null|  09| 大正|      532224|    514255|  栃木県|   1920| 1046479|     9|
|null|  10| 大正|      538504|    514106|  群馬県|   1920| 1052610|     9|
|nul

# SELECT INSERT
既に存在するテーブルに対して、検索結果をもとにデータを登録していくことができます。　　
SELECT INSERTの場合は、ADD PARTITIONは不要です

CTASと違うのはこちらはテーブルを作る操作ではなくて既にあるテーブルに対してデータを登録することが目的です。

In [98]:
# 今回は各地域のデータの履歴をまとめて一つのパーティションに入れるSelect Insertを記載してましょう。
spark.sql(""" 
Insert overwrite table super_crush_course.parquet_table_with_partition PARTITION(kenmei='all')

select code,gengo,wareki,seireki,chu,sokei,jinko_male,jinko_femail
 from 
super_crush_course.parquet_table_with_no_partition

""")

22/01/06 03:07:41 WARN log: Updating partition stats fast for: parquet_table_with_partition
22/01/06 03:07:41 WARN log: Updated size to 22841


DataFrame[]

In [100]:
spark.sql("show partitions super_crush_course.parquet_table_with_partition").show()

+-----------------------------+
|                    partition|
+-----------------------------+
|         kenmei=__HIVE_DEF...|
|                   kenmei=all|
|                kenmei=三重県|
|                kenmei=京都府|
|          kenmei=人口集中地区|
|kenmei=人口集中地区以外の地区|
|                kenmei=佐賀県|
|                  kenmei=全国|
|                kenmei=兵庫県|
|                kenmei=北海道|
|                kenmei=千葉県|
|              kenmei=和歌山県|
|                kenmei=埼玉県|
|                kenmei=大分県|
|                kenmei=大阪府|
|                kenmei=奈良県|
|                kenmei=宮城県|
|                kenmei=宮崎県|
|                kenmei=富山県|
|                kenmei=山口県|
+-----------------------------+
only showing top 20 rows



# INSERT/UPDATE/DELETE?
ビッグデータの世界では原則としてACIDをサポートしていません。  
そのため、UPDATEやDELETがサポートされていないことが多いです。

INSERTは単体で利用することはできますが、あまり出番がなく前述で紹介したSELECT INSERTでの出番が大半です。

# 分析関数の練習をしよう
ここからは、分析関数の練習をしてみましょう。  

- agg(groupby,count,sum)
- window(over)
- ピボットテーブル
- lag関数

データフレームでの操作とSQLでの操作を対比させながら実行していきます。

In [ ]:
# agg
# データを溜め込んでいくこと

# ピボットテーブル

# LAG関数

# データフレーム限定 RDDによる一行づつの操作

出番がある様な、無い様な操作ですが一つSparkの特徴であるRDD(低レベル操作)についてみていきましょう。  
Sparkは全てのDataFrameは実行されるときにRDDに変換されて実行されます(そのときに最適なRDD操作に変換してくれる)。

あまり普段RDDを意識することなく操作を可能です。

一応RDDでの操作も見ておきましょう。

RDDに変換すると、mapとかlambdaなどPythonの関数が適用になりますが、最適化をやってくれなくなるのであまりおすすめはできないです。

In [ ]:
rdds=json_df.rdd.map(lambda x: len(x.code))
rdds.take(10)

rdds.reduce(lambda a,b: a+b)

In [ ]:
spark.stop()